### 1

In [6]:
import pandas as pd
df = pd.read_csv("basic1.csv")

# print(df.describe())

Q1 = df['age'].quantile(.25)
Q3 = df['age'].quantile(.75)

result = Q3 - Q1

print(round(result))

50


In [7]:
import pandas as pd
df = pd.read_csv("fb.csv")

df['tmp'] = (df['loves'] + df['wows']) / df['reactions']

cond1 = df['tmp'] > 0.4
cond2 = df['tmp'] < 0.5
cond3 = df['type'] == 'video'

result = len(df[cond1 & cond2 & cond3])

print(result)

90


In [17]:
import pandas as pd
df = pd.read_csv("nf.csv")

# print(help(pd.DataFrame.dropna))

# print(df.shape)
df = df.dropna(subset=['date_added'])
# print(df.shape)

# print(df['date_added'].head(3)) # September 25, 2021

df['date_added'] = pd.to_datetime(df['date_added'])

# print(df['date_added'].head(3)) # 2021-09-25

# df['year'] = df['date_added'].dt.year.astype('string')
# df['month'] = df['date_added'].dt.month.astype('string')

# cond1 = df['year'] == 2018
# cond2 = df['month'] == 1
# cond3 = df['country'] == 'United Kingdom'

result = len(df[df['date_added'].astype('string').str.contains('2018-01') & cond3])

print(result)

6


<ipython-input-17-21aae46c3071>:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = len(df[df['date_added'].astype('string').str.contains('2018-01') & cond3])


In [16]:
import pandas as pd
df = pd.read_csv("nf.csv")

df['date_added'] = pd.to_datetime(df['date_added'])

df['year'] = df['date_added'].dt.year
df['month'] = df['date_added'].dt.month

cond1 = df['year'] == 2018
cond2 = df['month'] == 1
cond3 = df['country'] == 'United Kingdom'

result = df[cond1 & cond2 & cond3]

print(len(result))

6


### 2

In [5]:
# 1. 데이터 불러오기
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(train.shape, test.shape)

# 2. EDA
# print(train.info()) # float64(2), int64(3), object(6)
# print(test.info()) # float64(2), int64(2), object(6)

# print(train.isnull().sum())
# print(test.isnull().sum())

# print(train.head()) # ID 삭제하기
# print(test.head()) # ID pop

# print(train.describe(include='O'))
# print(test.describe(include='O'))


cols = train.select_dtypes(include='O').columns

# for col in cols:
#     A = set(train[col])
#     B = set(test[col])
#     print(A-B)
#     print(B-A)

# 3. 전처리 FE
target = train.pop('Segmentation')

train = train.drop('ID', axis=1)
test_id = test.pop('ID')

# obejct 삭제
# train = train.drop(cols, axis=1)
# test = test.drop(cols, axis=1)

# 원핫
# train = pd.get_dummies(train)
# test = pd.get_dummies(test)

# label
from sklearn.preprocessing import LabelEncoder

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

print(train.shape, test.shape)
# print(train.head())

# 4. train val split
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=123)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

# 5. 모델 선택 학습 예측 classifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=123)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

# print(pred)

# 6. 평가  f1_score macro
from sklearn.metrics import f1_score

print(f1_score(y_val, pred, average='macro'))
# print(help(f1_score))

# baseline 0.3808680264315276
# 원핫 0.4630094069778344
# lable 0.48123537267661165

# 7. test 예측 및 파일 제출
pred = model.predict(test)

submit = pd.DataFrame({
    'ID': test_id,
    'Segmentation': pred
})

submit.to_csv('submission.csv', index=False)

print(pd.read_csv('submission.csv').shape)

(6665, 11) (2154, 10)
(6665, 9) (2154, 9)
(5332, 9) (1333, 9) (5332,) (1333,)
0.48123537267661165
(2154, 2)
